## Problem
- we work at a bank and we have a HUGE dict mapping client names to their accounts balance
- we would like to expose that dict to some function with read-only access to the information.
- How to accomplish that in without compromising neither memory-efficiency nor security?

In [27]:
# for the sake of this recipe let's assume this dict was huge with millions of records and took lots of space in memory.
clients_balance = {'Junior': -50., 'Amy': 300., 'Zak': 500., 'Will': 0., 'Emilie': 1000.}

def send_reminder(clients_balance, threshold=0.):    
    for client, balance in clients_balance.items():   
        if balance <= threshold:
            print(f'\t Sending [REMINDER] to {client}: balance={balance} <= threshold={threshold}')
    
    ### MALICIOUS CODE: ####
    # Junior works at the bank and has an account,
    # He injects some malicious instructions to credit his account in a function meant for read-only purpose
    if clients_balance['Junior'] <= threshold:
        clients_balance['Junior'] += 1000.

## Answer
- the best way is to use: from types import MappingProxyType

In [28]:
# BAD WAY 1: Compromising security
clients_balance = {'Junior': -50., 'Amy': 300., 'Zak': 500., 'Will': 0., 'Emilie': 1000.}

print(f'[BEFORE send_reminder #1] balances={clients_balance}')
send_reminder(clients_balance) #<0> send_reminder can modify client_balances
print(f'[AFTER send_reminder #1] balances={clients_balance}') 

print('---')

print(f'[BEFORE send_reminder #2] balances={clients_balance}')
clients_balance['Amy'] = -10.
send_reminder(clients_balance) #<1> send_reminder no longer sends reminder to Junior
print(f'[AFTER send_reminder #2] balances={clients_balance}') 

[BEFORE send_reminder #1] balances={'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
	 Sending [REMINDER] to Junior: balance=-50.0 <= threshold=0.0
	 Sending [REMINDER] to Will: balance=0.0 <= threshold=0.0
[AFTER send_reminder #1] balances={'Junior': 950.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
---
[BEFORE send_reminder #2] balances={'Junior': 950.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
	 Sending [REMINDER] to Amy: balance=-10.0 <= threshold=0.0
	 Sending [REMINDER] to Will: balance=0.0 <= threshold=0.0
[AFTER send_reminder #2] balances={'Junior': 950.0, 'Amy': -10.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}


In [29]:
# BAD WAY 2: Compromising memory-efficiency
clients_balance = {'Junior': -50., 'Amy': 300., 'Zak': 500., 'Will': 0., 'Emilie': 1000.}

print(f'[BEFORE send_reminder #1] balances={clients_balance}')
balance_copy = dict(clients_balance)
send_reminder(balance_copy) #<2> send_reminder can not modify client_balances since we are passing a copy which is modified by send_reminder but not the real balance mapping.
print(f'[AFTER send_reminder #1] balances={clients_balance}') 

print('---')

print(f'[BEFORE send_reminder #2] balances={clients_balance}')
clients_balance['Amy'] = -10.
balance_copy = dict(clients_balance)
send_reminder(balance_copy) #<3> send_reminder still sends reminder to Junior
print(f'[AFTER send_reminder #2] balances={clients_balance}')

[BEFORE send_reminder #1] balances={'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
	 Sending [REMINDER] to Junior: balance=-50.0 <= threshold=0.0
	 Sending [REMINDER] to Will: balance=0.0 <= threshold=0.0
[AFTER send_reminder #1] balances={'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
---
[BEFORE send_reminder #2] balances={'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
	 Sending [REMINDER] to Junior: balance=-50.0 <= threshold=0.0
	 Sending [REMINDER] to Amy: balance=-10.0 <= threshold=0.0
	 Sending [REMINDER] to Will: balance=0.0 <= threshold=0.0
[AFTER send_reminder #2] balances={'Junior': -50.0, 'Amy': -10.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}


In [33]:
# GOOD WAY: preserving both memory-efficiency and security
from types import MappingProxyType

clients_balance = {'Junior': -50., 'Amy': 300., 'Zak': 500., 'Will': 0., 'Emilie': 1000.}

print(f'[BEFORE send_reminder #1] balances={clients_balance}')
readonly_balance_copy = MappingProxyType(clients_balance) #<4> create a read-only copy but dynamic view of the original clients_balance
send_reminder(readonly_balance_copy) #<5> raises an exception since the malicious code is trying to edit the readonly_balance_copy


[BEFORE send_reminder #1] balances={'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
	 Sending [REMINDER] to Junior: balance=-50.0 <= threshold=0.0
	 Sending [REMINDER] to Will: balance=0.0 <= threshold=0.0


TypeError: 'mappingproxy' object does not support item assignment

In [34]:
print(f'readonly_balance_copy = {readonly_balance_copy}')
clients_balance['Amy'] = -10. #<6>
print(f'readonly_balance_copy = {readonly_balance_copy}') #<7> the changes on clients_balance are replicated on readonly_balance_copy dynamically.

readonly_balance_copy = {'Junior': -50.0, 'Amy': 300.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}
readonly_balance_copy = {'Junior': -50.0, 'Amy': -10.0, 'Zak': 500.0, 'Will': 0.0, 'Emilie': 1000.0}


## Discussion
- <0> send_reminder can modify client_balances since we are passing a dict which is a mutable object
- <1> send_reminder no longer sends reminder to Junior since the previous call credited his account of 1000. 
- <2> send_reminder can not modify client_balances since we are passing a copy which is modified by send_reminder but not the real balance mapping
- <3> send_reminder still sends reminder to Junior
- <4, 5> MappingProxyType creates a read-only dict from the original (ie: write operations raise exceptions)
- <6, 7> The dict created by MappingProxyType is read-only but also dynamic as the changes on the original one are replicated seemlessly, so it's not copying the values but just restricting some methods of the dict class: Providing both security and memory-efficiency.